In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
import h5py
from peakfinder import detect_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_labels = pd.read_csv(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Scripts\smooth_labels.dat',delimiter = '\s+',header = None,index_col = None)
#df_labels = pd.read_csv(r'/Users/kunal/OneDrive - The Open University/SPIN/Transmission/Scripts/smooth_labels.dat',delimiter = '\s+',header = None,index_col = None)

df_labels.columns = ['File','Label']

In [3]:
z = []
y = []
#Trans = pd.Series()
#Alt = pd.Series()
rootDir = r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\New occultations'
#rootDir = r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations'
os.chdir(rootDir)
list_of_files = os.listdir(os.getcwd())

for each_file in list_of_files:
    for each_df_file in df_labels['File']:
        if each_file.startswith(each_df_file):
            
            file = h5py.File(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\New occultations\%s' %each_file,'r')
            #file = h5py.File(r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations/%s' %each_file,'r')

            
            T = np.array(file['Science/Transmission'])
            TangAlt = np.array(file['Geometry/Point0/TangentAltSurface'])
            wav = np.array(file['Science/Wavelength'])

            avg_TangAlt = np.array([])

            for j in range(TangAlt.shape[0]):
                avg_TangAlt = np.append(avg_TangAlt,np.mean(TangAlt[j,:]))

            T_250 = T[:,np.array(np.where(wav == min(abs(wav-250))+250)).flatten()].reshape(-1,)
            #Trans[i] = T_250
            #Alt[i] = avg_TangAlt
            if T_250[0] > 0.5:
                T_250 = T_250[::-1]

            z.append(T_250)
            
            if avg_TangAlt[0] > 100:
                avg_TangAlt = avg_TangAlt[::-1]
            
            y.append(avg_TangAlt)

In [4]:
z = np.array(z)
y = np.array(y)

In [5]:
alt_interp = np.arange(0,274,1)

In [6]:
t_interp = []
for i in range(len(z)):
    z_interp = np.interp(alt_interp,y[i],z[i])
    t_interp.append(z_interp)
t_interp = np.array(t_interp)

In [7]:
t_clipped = []
for i,t in enumerate(t_interp):
    clip = np.array([])
    peak_idx = detect_peaks(t) 
    for _ in t[peak_idx]:
        clip = np.append(clip,_)
        if _ > 0.98:
            break
    t_clipped.append(t[:np.where(t == clip[-1])[0][0]])
t_clipped = np.array(t_clipped)

In [10]:
indices = np.arange(len(np.array(df_labels['Label'])))
X_train, X_val, y_train, y_val,id_train,id_test = train_test_split(t_interp, np.array(df_labels['Label']),indices, test_size=0.33)

In [14]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

Learning rate:  0.05
Accuracy score (training): 0.848
Accuracy score (validation): 0.714
Learning rate:  0.075
Accuracy score (training): 0.857
Accuracy score (validation): 0.714
Learning rate:  0.1
Accuracy score (training): 0.884
Accuracy score (validation): 0.714
Learning rate:  0.25
Accuracy score (training): 1.000
Accuracy score (validation): 0.786
Learning rate:  0.5
Accuracy score (training): 1.000
Accuracy score (validation): 0.732
Learning rate:  0.75
Accuracy score (training): 1.000
Accuracy score (validation): 0.732
Learning rate:  1
Accuracy score (training): 1.000
Accuracy score (validation): 0.786


In [15]:
gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_features=2, max_depth=2, random_state=0)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_val)

print("Confusion Matrix:")
print(confusion_matrix(y_val, predictions))

print("Classification Report")
print(classification_report(y_val, predictions))

Confusion Matrix:
[[24  7]
 [ 5 20]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.77      0.80        31
           1       0.74      0.80      0.77        25

    accuracy                           0.79        56
   macro avg       0.78      0.79      0.78        56
weighted avg       0.79      0.79      0.79        56



In [17]:
from xgboost import XGBClassifier

In [18]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [19]:
score = xgb_clf.score(X_val, y_val)
print(score)

0.8035714285714286
